## 라이브러리

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import csv
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'NanumGothic'

## vvt 파일 불러오기

In [5]:
import os
path = '../script'

# 다큐멘터리(0), 영화(1) 선택
category = 1
if category == 0:
    path = path + '/Documentary'
elif category == 1:
    path = path + '/Movie'

_list = os.listdir(path)
file_list = []
for file in _list:
    if file.startswith('.'): continue
    file_list.append(file)

file_list = sorted(file_list)
print(f'작품개수: {len(file_list)}')
# print([{i:_list[i]} for i in range(len(file_list))])
for index, title in enumerate(file_list):
    print(f'{index}: {title}')

작품개수: 41
0: About.Time
1: Arthur.Christmas
2: Baby.Driver
3: Bad.Boys
4: Bee.Movie
5: Ben.Platt.Live.from.Radio.City.Music.Hall
6: Constantine
7: Dolittle
8: Dracula.Untold
9: Enola.Holmes
10: How.to.Be.Single
11: In.Time
12: Inception
13: Interstellar
14: It_s.Complicated
15: Jumanji_.Welcome.to.the.Jungle
16: Kill.Bill_.Vol..1
17: Klaus
18: Kung.Fu.Panda
19: Kung.Fu.Panda.2
20: Kung.Fu.Panda.3
21: Madagascar_.Escape.2.Africa
22: Matilda
23: Monty.Python.and.the.Holy.Grail
24: Morning.Glory
25: Notting.Hill
26: Passengers
27: Penguins.of.Madagascar_.The.Movie
28: Prisoners
29: Second.Act
30: The.Accountant
31: The.Boy.in.the.Striped.Pajamas
32: The.Dark.Knight
33: The.Departed
34: The.Family.Man
35: The.Intern
36: The.Judge
37: The.Revenant
38: The.Truman.Show
39: There.Will.Be.Blood
40: Zodiac


## 여기서 작품 선택합니다 꼭!!

In [6]:
# 바로 위 코드에서 file_list를 참고하여 작품 선택
title = file_list[22]

file_name = title + '.WEBRip.Netflix.en[cc]'
print(f'{path}/{title}/{file_name}')

../script/Movie/Matilda/Matilda.WEBRip.Netflix.en[cc]


## VTT 파일 읽기

In [7]:
opened_file = open(f'{path}/{title}/{file_name}.vtt', encoding='utf8')
content = opened_file.read()
segments = content.split('\n\n')    # 한줄씩 정리

In [8]:
segments[:5]

['WEBVTT',
 'NOTE Netflix\nNOTE Profile: webvtt-lssdh-ios8\nNOTE Date: 2016/05/11 00:06:24',
 'NOTE SegmentIndex\nNOTE Segment=595.845 48648@599 295\nNOTE Segment=596.471 52050@49247 323\nNOTE Segment=586.920 58741@101297 361\nNOTE Segment=597.555 53895@160038 333\nNOTE Segment=594.511 52101@213933 321\nNOTE Segment=595.678 49750@266034 307\nNOTE Segment=597.555 34205@315784 212\nNOTE Segment=585.919 29290@349989 178\nNOTE Segment=597.222 40397@379279 252\nNOTE Segment=459.250 20655@419676 125\nNOTE /SegmentIndex',
 '                                                                          ',
 '                    \n1\n00:00:21.313 --> 00:00:23.273  position:44.29%,start  align:start size:19.52%  line:84.62% \n<c.white><c.mono_sans>[♪♪♪]</c.mono_sans></c.white>']

## 불필요한 부분 제거

In [9]:
m = re.compile(r"\<.*?\>")
def clean(content):
    if '♪' in content: return None
    if '[' in content: return None
    if ']' in content: return None
    if '(' in content: return None
    if ')' in content: return None
    if '{' in content: return None
    if '}' in content: return None
    new_content = m.sub('',content)
    new_content = new_content.replace('-->','')
    remove_point_1 = new_content.find('position:')
    remove_point_2 = new_content.find('% \n')
    new_content = new_content[:remove_point_1] + new_content[remove_point_2+3:]
    return new_content

In [10]:
new_segments = [clean(s) for s in segments if len(s)!=0][4:]
new_segments[:5]

[None,
 '2\n00:00:43.126  00:00:45.128  NARRATOR:',
 '3\n00:00:43.126  00:00:45.128  Everyone is born,',
 '4\n00:00:45.128  00:00:47.923  but not everyone',
 '5\n00:00:45.128  00:00:47.923  is born the same.']

## 줄번호 / 속도 시작 / 속도 끝 / 자막 - 데이터 분리

In [11]:
trimmed_segments = []
for segment in new_segments:
    if segment != None and segment.strip() != '':
      split_segment = segment.split()
      times = split_segment[:3]
      script = ' '.join(split_segment[3:])
      times.append(script)
      trimmed_segments.append(times)

In [12]:
trimmed_segments[:5]

[['2', '00:00:43.126', '00:00:45.128', 'NARRATOR:'],
 ['3', '00:00:43.126', '00:00:45.128', 'Everyone is born,'],
 ['4', '00:00:45.128', '00:00:47.923', 'but not everyone'],
 ['5', '00:00:45.128', '00:00:47.923', 'is born the same.'],
 ['6', '00:00:47.923', '00:00:50.592', 'Some will grow']]

## 데이터 저장

In [13]:
with open(f'{path}/{title}/{file_name}.csv', 'w', encoding='utf8', newline='') as f:
    for line in trimmed_segments:
        thewriter = csv.writer(f)
        thewriter.writerow(line)

## 최종 - 줄번호 / 속도 시작 / 속도 끝 / 대사 별로 컬럼 분리

In [14]:
pd.set_option('display.max_colwidth', None)
data = pd.read_csv(f'{path}/{title}/{file_name}.csv', names=['num', 'start', 'end', 'script'])

display(len(data), data.tail())


2400

,num,start,end,script
2395,2701,01:32:27.667,01:32:29.503,"""never mind how long"
2396,2702,01:32:27.667,01:32:29.503,"precisely,"
2397,2703,01:32:29.503,01:32:32.047,"""having little or no"
2398,2704,01:32:29.503,01:32:32.047,"money in my purse,"
2399,2705,01:32:32.047,01:32:34.799,"and nothing particular to..."""


In [15]:
if data['script'].iloc[-1][-1] not in ['.', '!', '?']:
    data['script'].iloc[-1] = data['script'].iloc[-1] + '.'

data.tail()

,num,start,end,script
2395,2701,01:32:27.667,01:32:29.503,"""never mind how long"
2396,2702,01:32:27.667,01:32:29.503,"precisely,"
2397,2703,01:32:29.503,01:32:32.047,"""having little or no"
2398,2704,01:32:29.503,01:32:32.047,"money in my purse,"
2399,2705,01:32:32.047,01:32:34.799,"and nothing particular to...""."


In [16]:
data['script'] = data['script'].str.replace('\"', '')

# 쪼개진 문장 합치기
for index, current_row in data.iterrows():
  if current_row['script'][-1] in ['.', '!', '?']:
    continue
  else:
    row_index = index + 1
    while True:
      next_row = data.loc[row_index]
      new_script = data.loc[index, 'script'] + ' ' + next_row['script']
      data.loc[index, 'script'] = new_script
      if next_row['script'][-1] in ['.', '!', '?']:
        data.loc[index, 'end'] = next_row['end']
        break
      else:
        row_index += 1
print(row_index)
# 필요없는 토막문장들 제거
data = data.drop_duplicates(subset='end')
data.head(30)

2399


,num,start,end,script
0,2,00:00:43.126,00:00:47.923,"NARRATOR: Everyone is born, but not everyone is born the same."
4,6,00:00:47.923,00:00:52.553,Some will grow to be butchers or bakers or candlestick makers.
7,9,00:00:52.553,00:00:56.264,Some will only be really good at making Jell-O salad.
9,11,00:00:56.264,00:01:00.602,"One way or another, though, every human being is unique..."
11,13,00:01:00.602,00:01:02.897,for better or for worse.
12,15,00:01:04.231,00:01:09.069,Most parents believe their children are the most beautiful creatures ever to grace the planet.
16,19,00:01:09.069,00:01:11.030,Others take a less emotional approach.
18,21,00:01:11.030,00:01:13.240,What a waste of time.
20,23,00:01:13.240,00:01:15.951,"And expensive, $9.25 for a bar of soap!"
22,25,00:01:15.951,00:01:17.745,"Well, I had to take a shower, Harry."


In [17]:
data.to_csv(f'{path}/{title}/{file_name}.csv', columns=['start', 'end', 'script'], index=False)
data = pd.read_csv(f'{path}/{title}/{file_name}.csv')

data.head()

,start,end,script
0,00:00:43.126,00:00:47.923,"NARRATOR: Everyone is born, but not everyone is born the same."
1,00:00:47.923,00:00:52.553,Some will grow to be butchers or bakers or candlestick makers.
2,00:00:52.553,00:00:56.264,Some will only be really good at making Jell-O salad.
3,00:00:56.264,00:01:00.602,"One way or another, though, every human being is unique..."
4,00:01:00.602,00:01:02.897,for better or for worse.
